In [1]:
from tqdm import tqdm
import pickle
import facebook_scraper as fs
import random
import time
import os

/Users/nathanmalta/miniconda3/envs/twitter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### First, grab the last 400 posts without comments

This can be done quickly, and we can use the post urls to restart scraping where we left off if there is a problem

In [4]:
# number of comments to download -- set this to True to download all comments
MAX_COMMENTS = 100
NUM_POSTS = 400

airline = 'SpiritAirlines' #or 'SouthwestAir' or 'AmericanAirlines' or 'Delta'

#enter a facebook account email and password (use a fake account as there's a chance it could get banned)
account_email = ''
account_password = ''

# get the post (this gives a generator)
gen = fs.get_posts(
    account=airline, 
    pages=50,
    options={"comments": MAX_COMMENTS, "progress": True, 'allow_extra_requests': False, 'posts_per_page': 100}, 
    credentials=(account_email, account_password)
)

# take 1st element of the generator which is the post we requested
posts = []
for _ in tqdm(range(NUM_POSTS)):
    new = next(gen)
    posts.append(new)

/Users/nathanmalta/miniconda3/envs/twitter/lib/python3.11/site-packages/facebook_scraper/facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")
  0%|          | 0/400 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# save posts
with open('posts_no_com/spirit.pkl', 'wb') as f:
    pickle.dump(posts, f)

In [ ]:
#load posts
with open('posts_no_com/spirit.pkl', 'rb') as f:
    posts = pickle.load(f)

In [ ]:
print(len(posts))

### Then, start scraping the posts for comments

In [ ]:
MAX_COMMENTS = 50
NUM_POSTS = 400

posts_with_comments = []

if os.path.exists('posts_with_com/spirit/spirit.pkl'):
    with open('posts_with_com/spirit/spirit.pkl', 'rb') as f:
        posts_with_comments = pickle.load(f)

In [ ]:
#start (or restart) downloading comments
while len(posts_with_comments) < NUM_POSTS:
    print(len(posts_with_comments))
    post = posts[len(posts_with_comments)]
    url = post['post_url']
    gen = fs.get_posts(
        post_urls=[url],
        options={"comments": MAX_COMMENTS, "progress": True, 'allow_extra_requests': False, 'posts_per_page': 100}, 
        credentials=(account_email, account_password)
        # cookies='cookies.json'
    )
    posts_with_comments.append(next(gen))
    sleep_time = random.randint(1, 5)
    time.sleep(sleep_time)

In [ ]:
#save downloaded posts + comments to a file
with open('scraped_data/spirit.pkl', 'wb') as f:
    pickle.dump(posts_with_comments, f)

In [ ]:
#view a post and its comments
print('num posts', len(posts_with_comments))
post_num = 399
print(posts_with_comments[post_num]['text'])
print(f'num comments: {len(posts_with_comments[post_num]["comments_full"])}')
for comment in posts_with_comments[post_num]['comments_full']:
    print(comment['comment_text'])

print(posts_with_comments[post_num]['time'])